In [1]:
import torch.nn as nn

## 1.构建模型

In [4]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        #conv1: 16 * 5 * 5保持宽高的卷积(stride=1, padding=2), relu，2 * 2的maxpool
        self.conv1=nn.Sequential(
            #卷积函数
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1,padding=2),#===>[16,28,28]
            nn.ReLU(),                                                                  #===>[16,28,28]
            #池化函数
            nn.MaxPool2d(kernel_size=2)                                                 #===>[16,14,14]
        )
        #conv2:32*5*5保持宽高的卷积(stride=1, padding=2), relu，2 * 2的maxpool
        self.conv2=nn.Sequential(
            nn.Conv2d(16,32,5,1,2),                                                     #===>[32,14,14]
            nn.ReLU(),                                                                  #===>[32,14,14]
            nn.MaxPool2d(kernel_size=2)                                                 #===>[32,7,7]
        )
        #全连接层：[32*7*7,10]
        self.out=nn.Linear(32*7*7,10)

    #重写forward()
    def forward(self, x):
        x=self.conv1(x)

        x=self.conv2(x)
        print(x.shape)
        #全连接之前，展平,保留batch的维度[batch,32,7,7]==>[batch,32*7*7]
        x=x.view(x.size(0),-1)#x.size(0)指batchsize的值,而-1指在不告诉函数有多少列的情况下，根据原tensor数据和batchsize自动分配列数
        output=self.out(x)
        return output,x

#8.搭建CNN
model=CNN()

## 2.使用初始化函数初始化参数

遍历模型的每1层，根据网络层的不同定义不同的初始化方式

In [ ]:
def weight_init_1(model):
    for m in model.modules:
        #Linear层初始化
        if isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)
        # 也可以判断是否为conv2d，使用相应的初始化方式 
        elif isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
         # 是否为批归一化层
        elif isinstance(m, nn.BatchNorm2d):
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)

## 3.从文件中加载预训练模型保存的参数

1. 从文件路径中反序列化预训练参数字典:torch.load()
2. 获取当前模型的参数字典:model.state_dict()
3. 在预训练参数字典中保留当前模型存在的参数
4. 更新当前模型参数字典
5. 当前模型加载更新后的参数字典

In [ ]:
def weight_init_2(model,pretrain_path):
    
    #加载预训练模型的参数
    if os.path.isfile(pretrain_path):
        #先反序列化预训练模型保存的预训练参数字典
        pretrained_dict = torch.load(pretrain_path)
        #当前模型所有参数的字典
        model_dict = model.state_dict()
        #在预训练参数字典中保留当前模型存在的参数
        pretrained_dict = {k: v for k, v in pretrained_dict.items()
                           if k in model_dict.keys()}
        #更新当前模型参数字典
        model_dict.update(pretrained_dict)
        #加载
        model.load_state_dict(model_dict)

## Example

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        #conv1: 16 * 5 * 5保持宽高的卷积(stride=1, padding=2), relu，2 * 2的maxpool
        self.conv1=nn.Sequential(
            #卷积函数
            nn.Conv2d(in_channels=3,out_channels=16,kernel_size=5,stride=1,padding=2),#===>[16,28,28]
            nn.ReLU(),                                                                  #===>[16,28,28]
            #池化函数
            nn.MaxPool2d(kernel_size=2)                                                 #===>[16,14,14]
        )
        #conv2:32*5*5保持宽高的卷积(stride=1, padding=2), relu，2 * 2的maxpool
        self.conv2=nn.Sequential(
            nn.Conv2d(16,32,5,1,2),                                                     #===>[32,14,14]
            nn.ReLU(),                                                                  #===>[32,14,14]
            nn.MaxPool2d(kernel_size=2)                                                 #===>[32,7,7]
        )
        #全连接层：[32*7*7,10]
        self.out=nn.Linear(32*7*7,10)

    #重写forward()
    def forward(self, x):
        x=self.conv1(x)

        x=self.conv2(x)
        print(x.shape)
        #全连接之前，展平,保留batch的维度[batch,32,7,7]==>[batch,32*7*7]
        x=x.view(x.size(0),-1)#x.size(0)指batchsize的值,而-1指在不告诉函数有多少列的情况下，根据原tensor数据和batchsize自动分配列数
        output=self.out(x)
        return output,x
    
        def init_weights(self, pretrained='',):
        logger.info('=> init weights from normal distribution')
        #参数初始化
        for m in self.modules():
            #Conv2d层参数初始化:正态分布weight
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight, std=0.001)
            #BatchNorm层参数初始化:常量weight=1,bias=0
            elif isinstance(m, InPlaceABNSync):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
        #加载预训练模型的参数
        if os.path.isfile(pretrained):
            #先反序列化预训练模型保存的预训练参数字典
            pretrained_dict = torch.load(pretrained)
            logger.info('=> loading pretrained model {}'.format(pretrained))
            #当前模型所有参数的字典
            model_dict = self.state_dict()
            #在预训练参数字典中保留当前模型存在的参数
            pretrained_dict = {k: v for k, v in pretrained_dict.items()
                               if k in model_dict.keys()}
            for k, _ in pretrained_dict.items():
                logger.info(
                    '=> loading {} pretrained model {}'.format(k, pretrained))
            #更新当前模型参数字典
            model_dict.update(pretrained_dict)
            #加载
            self.load_state_dict(model_dict)